In [1]:
from harness import Harness

import sklearn.datasets
import sklearn.discriminant_analysis
import sklearn.mixture
import sklearn.model_selection

from pandas import ( 
    CategoricalIndex, DataFrame, Index, Series
)

from IPython.display import (
    Markdown,
)


In [2]:
"""Can I load the iris data"""
iris = sklearn.datasets.load_iris()

In [3]:
"""Can I initialize a Harness DataFrame"""
target = CategoricalIndex(iris['target']).rename('target')
df = Harness(
    data=iris['data'], 
    index=target,
    columns=iris['feature_names'],
    estimator=sklearn.discriminant_analysis.LinearDiscriminantAnalysis(),
    feature_level='target',
)

df = df.set_index(
    df.index
    .rename_categories(iris['target_names'])
    .rename('target_name'), append=True
).set_index(
    df
    .index.rename_categories(['red', 'green', 'blue'])
    .rename('color'), append=True
)

df.sample(5)

sepal length (cm)  sepal width (cm)  \
target target_name color                                        
2      virginica   blue                 6.1               2.6   
0      setosa      red                  4.4               2.9   
                   red                  5.4               3.9   
1      versicolor  green                5.6               2.5   
0      setosa      red                  4.8               3.0   

                          petal length (cm)  petal width (cm)  
target target_name color                                       
2      virginica   blue                 5.6               1.4  
0      setosa      red                  1.4               0.2  
                   red                  1.7               0.4  
1      versicolor  green                3.9               1.1  
0      setosa      red                  1.4               0.1

In [4]:
test_train = next(
    sklearn.model_selection
    .StratifiedKFold(n_splits=2)
    .split(df.values, df.Index('target'))
)

split = DataFrame(index=df.index, columns=['split'])
split.iloc[test_train[0]], split.iloc[test_train[1]] = 'train', 'test'
df = df.set_index(split.set_index('split', append=True).swaplevel(-1,0).index)

df.ix['train'].fit().sample(5)

,,,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
target_name,color,target,,,,
versicolor,green,1,6.6,3.0,4.4,1.4
setosa,red,0,4.8,3.1,1.6,0.2
virginica,blue,2,6.2,3.4,5.4,2.3
setosa,red,0,4.7,3.2,1.6,0.2
versicolor,green,1,6.1,3.0,4.6,1.4


In [5]:
df.ix['train'].score()

0.97333333333333338

In [6]:
df.add_template(
scoreboard="""
The table presents some information about the `{{
    df.estimator.__str__().split('(',1)[0]
}}` model applied to the `iris` dataset.

|         |           Test          |           Training       |
|---------|-------------------------|--------------------------|
| Score   |{{df.ix['test'].score()}}|{{df.ix['train'].score()}}|
| Samples | {{df.ix['test'] | len}} | {{df.ix['train'] | len}} |
"""
);

In [7]:
Markdown(df.get_template('scoreboard'))


The table presents some information about the `LinearDiscriminantAnalysis` model applied to the `iris` dataset.

|         |           Test          |           Training       |
|---------|-------------------------|--------------------------|
| Score   |0.986666666667|0.973333333333|
| Samples | 75 | 75 |

In [8]:
transformed = df.transform()
transformed.sample(2)

0         1
split target_name color target                    
train versicolor  green 1      -4.858767 -1.157171
      setosa      red   0       7.341003 -0.791289

In [9]:
transformed = transformed.set_index(
    target.rename_categories(["🌸", "🌺", "🌼"]).rename('emojis'),
    append=True,
)

In [10]:
with transformed.reset_index().DataSource(
    x=0, y=1, fill_color='color', text='emojis'
) as source:
    (source.output_file("iris_emoji.html")
         .figure(responsive=True)
         .ColumnDataSource()
         .X(size=20, line_alpha=.6)
         .Text(
            text_align='center', text_baseline='middle', 
            text_font_size="40px", text_alpha=.6
          )
         .Circle(size=3)
         .save()
    )